In [135]:
from __future__ import print_function
import time

import sys
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,UpSampling2D,Permute,Reshape,Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras import backend as K
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
img_rows = 512
img_cols = 512

In [149]:
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(n_classes, (1, 1), activation='softmax')(conv9)
    

    model.compile(optimizer=Adam(lr=1e-3), loss="categorical_crossentropy")

    return model

In [150]:
num_images=1
n_classes=3
def preprocess_img(imgs):
    imgs_p = np.ndarray((num_images, img_rows, img_cols), dtype=np.uint8)
    for i in range(num_images):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True, mode="constant")
        #pass
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

def preprocess_mask(imgs):
    imgs_p = np.ndarray((num_images, img_rows, img_cols,n_classes), dtype=np.uint8)
    #imgs_p = imgs_p[..., np.newaxis]
    return imgs_p


In [151]:
imgs_train = np.load('imgs_train.npy').astype('float32')
imgs_mask_train = np.load('imgs_mask_train.npy').astype('float32')
imgs_train /= 255
imgs_train=to_categorical(imgs_train, num_classes=3)
print (imgs_mask_train.shape)

imgs_train=preprocess_img(imgs_train)
imgs_mask_train=preprocess_mask(imgs_mask_train)


(512, 512, 3)


C:\Users\Moses\Anaconda3\envs\tf_biobsy\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [155]:
print (imgs_train.shape)
print (imgs_mask_train.shape)

(1, 512, 512, 1)
(1, 512, 512, 3)


In [156]:
model=get_unet()

In [157]:
model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=1, verbose=1, shuffle=True,
              validation_split=0)

ValueError: Error when checking target: expected reshape_23 to have 3 dimensions, but got array with shape (1, 512, 512, 3)

In [158]:
imgs_test=imgs_train
imgs_mask_test = model.predict(imgs_test, verbose=1,batch_size=1)
np.save('imgs_mask_test.npy', imgs_mask_test)
print(imgs_mask_test.shape)


1/1 [==============================] - 6s 6s/step
(1, 3, 262144)


In [159]:
result = imgs_mask_test[0]
print (result)

[[0.5 0.5 0.5 ... 0.5 0.5 0.5]
 [0.5 0.5 0.5 ... 0.5 0.5 0.5]
 [0.5 0.5 0.5 ... 0.5 0.5 0.5]]
